In [25]:
import numpy as np

In [151]:
import pickle

with open('mnist.pkl', 'rb') as f:
    mnist = pickle.load(f)
X = mnist['X'][:10000].astype('float32')
Y = mnist['Y'][:10000].astype('float32')
X -= X.mean(axis=0)
U, S, VT = np.linalg.svd(X.T @ X)
X = X @ VT[:100].T
X /= X.std(axis=0)
Y /= Y.std(axis=0)

In [172]:
C31 = Y.T @ X / len(X)

In [176]:
from functools import reduce

def forward(X, Ws):
    return X @ reduce(np.dot, Ws[::-1]).T

def loss(X, Y, Ws):
    return np.mean((Y - forward(X, Ws)) ** 2)

In [197]:
def grad(C31, Ws):
    """
    E = Y - forward(X, Ws)
    X.T @ E = X.T @ Y - X.T @ forward(X, Ws)
            = C13 - X.T @ X @ Ws.T
            = C31 - Ws @ X.T @ X
            = C31 - Ws if C11 = I
            = S31 - Wbars under change of variables
    """
    dg = reduce(np.dot, Ws[::-1]) - C31
    dW3 = (W2 @ W1 @ dg.T).T
    dW2 = (W1 @ dg.T @ W3).T
    dW1 = (dg.T @ W3 @ W2).T
    return dW1, dW2, dW3

In [198]:
W1 = np.random.randn(40, 100) / np.sqrt(100)
W2 = np.random.randn(20, 40) / np.sqrt(40)
W3 = np.random.randn(10, 20) / np.sqrt(20)

loss(X, Y, [W1, W2, W3])

2.494201724897844

In [204]:
grads = grad(C31, [W1, W2, W3])
for dW, W in zip(grads, [W1, W2, W3]):
    W -= 0.001 * dW

In [205]:
loss(X, Y, [W1, W2, W3])

2.4293152254819312

In [206]:
def output_modes(U33, W32):
    return U33.T @ W32

def input_modes(V11, W21):
    return V11 @ W21.T

In [ ]:
output_modes(